# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [21]:
import pandas as pd
DATASET_ENCODING = "ISO-8859-1"
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
data = pd.read_csv("tweets.csv", encoding =DATASET_ENCODING , names=DATASET_COLUMNS)
data = data.sample(n=10000)
data.shape

(10000, 6)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [22]:
def clean_up(s):
    import re
    s=s.replace("""'""",' ')
    s= re.sub(r'http\S+',' ',s)
    s=re.sub('[\d]',' ',s)
    s= re.sub('[^\w\s]',' ',s)
    s= re.sub('\s\s+',' ',s)
    s=s.lower().strip()
    return s
def tokenize(s):
    from nltk.tokenize import word_tokenize
    return word_tokenize(s)
def stem_and_lemmatize(l):
    from nltk import stem
    stemmer = stem.PorterStemmer()
    o=[]
    for b in l:
        o.append(stemmer.stem(b))
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    p=[]
    for a in o:
        p.append(lemmatizer.lemmatize(a))
    return p
def remove_stopwords(l):
    from nltk.corpus import stopwords
    p=[]
    stop_words = set(stopwords.words('english')) 
    for w in l: 
        if w not in stop_words: 
            p.append(w)
    return p
data['text_processed'] = data['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)
data.head()

,target,ids,date,flag,user,text,text_processed
895801,4,1692818014,Sun May 03 20:51:26 PDT 2009,NO_QUERY,PEARliExo3,"i swear lauren conrad's my best friendd , she...","[swear, lauren, conrad, best, friendd, know, y..."
276439,0,1991033037,Mon Jun 01 06:16:04 PDT 2009,NO_QUERY,chriswil16,really dislikes fevers/the chills,"[realli, dislik, fever, chill]"
67803,0,1692522377,Sun May 03 20:10:40 PDT 2009,NO_QUERY,PreciousCali1,whats up everyone in twitter nation ..just got...,"[everyon, twitter, nation, got, done, cook, di..."
129390,0,1835096405,Mon May 18 05:07:38 PDT 2009,NO_QUERY,lumpadoodles,hate public transport stood waiting over half...,"[hate, public, transport, stood, wait, half, h..."
345192,0,2015845584,Wed Jun 03 05:38:40 PDT 2009,NO_QUERY,JelleDeVroe,i think im gonna have a mental breakdown by th...,"[think, im, gon, na, mental, breakdown, end, w..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [23]:
data_processed = data['text_processed'].tolist()
data_processed = [item for sublist in data_processed for item in sublist]
from nltk.probability import FreqDist
word_count = FreqDist(data_processed)
top_fiveh = sorted(word_count, key=word_count.get, reverse=True)[:5000]

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [24]:
# your code here
def find_features(words):
    features = {}
    for w in top_fiveh:
        features[w] = (w in words)
    return features

In [25]:
featuresets = []
for index, row in data.iterrows():
    featuresets.append((find_features(row['text_processed']), row['target']==4))

In [26]:
featuresets[:5]

[({'go': False,
   'get': False,
   'day': False,
   'thi': False,
   'wa': False,
   'work': False,
   'like': False,
   'good': False,
   'love': False,
   'today': False,
   'u': False,
   'time': False,
   'quot': False,
   'got': False,
   'back': False,
   'lol': False,
   'thank': False,
   'one': False,
   'miss': False,
   'know': True,
   'want': False,
   'think': False,
   'see': False,
   'feel': False,
   'realli': False,
   'im': False,
   'amp': False,
   'night': False,
   'hope': False,
   'make': False,
   'watch': False,
   'need': False,
   'look': False,
   'well': False,
   'still': False,
   'new': False,
   'much': False,
   'na': False,
   'home': False,
   'last': False,
   'oh': False,
   'ha': False,
   'come': False,
   'twitter': False,
   'tomorrow': False,
   'bad': False,
   'wish': False,
   'great': False,
   'morn': False,
   'week': False,
   'haha': False,
   'sleep': False,
   'whi': False,
   'thing': False,
   'wait': False,
   'fun': False,
  

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [27]:
# your code here
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split
train, test = train_test_split(featuresets, test_size=0.4)
classifier = NaiveBayesClassifier.train(train)

In [28]:
classifier.show_most_informative_features()

Most Informative Features
                     sad = True            False : True   =     15.8 : 1.0
                     ugh = True            False : True   =     15.6 : 1.0
                   sweet = True             True : False  =     12.8 : 1.0
                    hate = True            False : True   =     12.8 : 1.0
                    sick = True            False : True   =     11.6 : 1.0
                    poor = True            False : True   =      9.4 : 1.0
                     die = True            False : True   =      8.7 : 1.0
                homework = True            False : True   =      8.7 : 1.0
                    sigh = True            False : True   =      8.3 : 1.0
                   award = True             True : False  =      8.0 : 1.0


In [29]:
import nltk
nltk.classify.accuracy(classifier, test)

0.71025

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here